In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
df = pd.read_csv('../input/rainfall in india 1901-2015.csv')
df.head()

In [ ]:
df.loc[df.SUBDIVISION == 'KERALA'].isnull().sum()

In [ ]:
def tidy(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df.year.astype(str)+'-'+df.month.astype(str)+'-'+df.day.astype(str),format='%Y-%b-%d')
    return df.set_index('date')
df_krl = (df[df.SUBDIVISION == 'KERALA']
          .loc[:,'YEAR':'DEC']
          .set_index('YEAR')
          .stack()
          .reset_index()
          .rename({'YEAR':'year','level_1':'month',0:'precip'},axis=1)
          .assign(day=1)
          .reindex(['year','month','day','precip'],axis=1)
          .pipe(tidy)
          .drop(['year','month','day'],axis=1)
         )
df_krl.tail()

In [ ]:
ax = df_krl['1990':'2010'].plot(figsize=(21,3),grid=True)
ax.grid(color='gray',linewidth=1,linestyle='--',alpha=0.7)
#ax.set_xticks(range(1900,2020,5));

In [ ]:
fft = np.fft.fft(df_krl['1990':'2010'].precip.values)
ffta = np.abs(fft)
freq = np.arange(12/(len(df_krl['1990':'2010'])),6+12/(len(df_krl['1990':'2010'])),12/(len(df_krl['1990':'2010'])))
ds = pd.Series(ffta[:len(freq)],index=freq)
ds.iloc[1:].plot(figsize=(18,3),grid=True);

In [ ]:
def append(df):
    df = df.copy()
    return df.append([df]*115,ignore_index=True)
df_clim = df_krl.groupby(df_krl.index.month).mean().reset_index(drop=True).pipe(append)
dfkc = df_krl.reset_index().assign(clim=df_clim).set_index('date')

In [ ]:
ax = dfkc.assign(anom=dfkc['precip']-dfkc['clim']).loc['2010':'2015',['precip','clim']].plot(figsize=(21,3),grid=True)
ax.grid(color='gray',linewidth=1,linestyle='--',alpha=0.7)

In [ ]:
dfkc.precip.plot(kind='box')